In [1]:
import datetime
import json

import numpy as np
from itertools import product
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from architectures.space_time.parametric_pooling_net_ordering import ParametricNetWithPoolingOrdered
from train_utils import train_model_regression
from evaluation_GTCNN import rNMSELoss, MSELossWithSparsityRegularizer, compute_iteration_rNMSE
from pred_utils import get_device, transform_data_to_all_steps_prediction, get_name_string, get_dataset, \
    get_MOLENE_dataset
from layers import CPGNN_ST, CPGNN_ST_v2, CPGNN_ST_v3, CITRUS, SGPModel
import networkx as nx
from Utilsss import get_evcs_evals

torch.cuda.current_device()

/home/aref/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


0

In [2]:
device = get_device(use_gpu=True)

Device selected: cuda:0


In [3]:
ds_folder = "./"
splits = [0.35, 0.15, 0.5]
obs_window = 10

In [4]:
data, steps_ahead, weighted_adjacency = get_MOLENE_dataset(
    ds_folder,
    splits=splits,
    obs_window=obs_window
)
N_spatial_nodes = weighted_adjacency.shape[0]
print(f"{N_spatial_nodes} nodes - {obs_window} observed timesteps - steps ahead: {steps_ahead}")



MOLENE is selected



Dataset path: ./dataset/processed/dataset_w=10_steps=[1, 2, 3, 4, 5]_splits=[0.35, 0.15, 0.5].pickle
32 nodes - 10 observed timesteps - steps ahead: [1, 2, 3, 4, 5]


In [5]:
# Get data (We do not need test data/labels here)
trn_data, val_data, tst_data, trn_labels, val_labels, tst_labels = transform_data_to_all_steps_prediction(data, node_first=True, device=device)

trn_data = trn_data.float()
val_data = val_data.float()
tst_data = tst_data.float()
trn_labels = trn_labels.float()
val_labels = val_labels.float()
tst_labels = tst_labels.float()

In [6]:
# obtain one-step labels for the training
one_step_trn_labels = trn_labels[:, 0, :]  # [batch x step-ahead x nodes]
one_step_val_labels = val_labels[:, 0, :]
print(one_step_trn_labels.shape, one_step_val_labels.shape)

today = datetime.datetime.now().strftime("%Y%m%d%H%M%S")


torch.Size([246, 32]) torch.Size([98, 32])


In [7]:
N_ITERATIONS = 10
num_epochs = 1500


learning_rate = 0.001
weight_decay = 0
batch_size = 64
patience = 150
factor = 0.9
lambda_value = 0.00025 # 0

not_learning_limit = 300

In [8]:
# CPGNN:
n_nodes = weighted_adjacency.shape[0]
M = obs_window
N = [n_nodes, M]
k = list(np.array(N)-2)
K_list = [30, 8]

In [9]:
adj = weighted_adjacency
Graph_List = [nx.from_numpy_array(np.array(adj)), nx.path_graph(obs_window)]

evecs, evals, L_list = get_evcs_evals(Graph_List, K_list)

for ii in range(len(evals)):
    evals[ii] = evals[ii].to(device)

[30, 8]
evecs.shape:,  torch.Size([32, 30])
evecs.shape:,  torch.Size([10, 8])
evecs_kron.shape:,  torch.Size([10, 8])


In [10]:
dim = 16
N_block = 3

In [11]:
res_dict = {
            'lr': learning_rate,
            'results': []
        }

for i in range(N_ITERATIONS):

    print(100*'*' + ' iter: ' + str(i) + 100*'*')
    one_step_gtcnn = CITRUS(input_size=1,
                                n_nodes=n_nodes,
                                horizon=1,
                                emb_size=dim,
                                hidden_size=dim,
                                rnn_layers=1,
                                gnn_kernel=1,
                                mass = torch.ones(np.prod(N)).to(device),
                                evals = evals,
                                evecs = torch.tensor(evecs).to(device),
                                C_width = dim,
                                N_block = N_block,
                                single_t = False,
                                use_gdc = [],
                                num_nodes = N,
                                last_activation=torch.nn.LeakyReLU(), 
                                mlp_hidden_dims=[dim, dim, dim, dim], 
                                dropout=False, 
                                with_MLP=False, 
                                diffusion_method='spectral', 
                                device = device,
                                graph_wise=False).to(device)
    print(one_step_gtcnn)

    model_parameters = filter(lambda p: p.requires_grad, one_step_gtcnn.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(f"Number of parameters: {params}")


    # name_string = get_name_string(
    #     obs_window,
    #     feat_per_layer, taps_per_layer, time_pooling_ratio_per_layer,
    #     pool_reach_per_layer, active_nodes_per_timestep_per_layer,
    #     weight_decay, cyclic, lambda_value, time_directed
    # )
    # log_dir = f"./runs_MOLENE_w={obs_window}/{today}_lr={learning_rate}_b={batch_size}_{name_string}"

    log_dir = f"./runs_MOLENE_w={obs_window}/{today}_lr={learning_rate}_b={batch_size}_CPGNN_dim={dim}_l={N_block}_eig1={K_list[0]}_eig2={K_list[1]}"

    ### TRAINING ###
    loss_criterion = MSELossWithSparsityRegularizer(one_step_gtcnn, lambda_value) #torch.nn.MSELoss() #
    #torch.nn.MSELoss()

    val_metric = rNMSELoss()

    optimizer = torch.optim.Adam(one_step_gtcnn.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=patience, factor=factor)

    best_model, best_epoch = train_model_regression(
        Iter = i,
        model=one_step_gtcnn,
        training_data=trn_data, validation_data=val_data,  # [n_samples x 1 x nodes x timesteps]
        single_step_trn_labels=one_step_trn_labels, single_step_val_labels=one_step_val_labels,  # [n_samples x spatial_nodes]
        num_epochs=num_epochs, batch_size=batch_size,
        loss_criterion=loss_criterion, optimizer=optimizer, scheduler=scheduler,
        val_metric_criterion=val_metric,
        log_dir=log_dir,
        not_learning_limit=not_learning_limit
    )


    rNMSE_dict, predictions_dict = compute_iteration_rNMSE(best_model, steps_ahead, tst_data, tst_labels,
                                                           device, verbose=False)

    res_dict['results'].append([round(l.item(), 4) for l in list(rNMSE_dict.values())])

    means = [round(el, 4) for el in np.average(res_dict['results'], axis=0)]
    stds = [round(el, 4) for el in np.std(res_dict['results'], axis=0)]
    res_dict['final_res'] = {
        'avg': means,
        'std': stds
    }

    with open(log_dir + '/results.json', 'w', encoding='utf-8') as f:
        json.dump(res_dict, f, ensure_ascii=False, indent=4)

    print(res_dict['results'])

**************************************************************************************************** iter: 0****************************************************************************************************
CITRUS(
  (node_embeddings): NodeEmbedding(n_nodes=32, embedding_size=16)
  (encoder): Linear(in_features=17, out_features=16, bias=True)
  (CPGNN): CPGNN_ST_in_TTS(
    (last_activation): LeakyReLU(negative_slope=0.01)
    (first_lin): Linear(in_features=16, out_features=16, bias=True)
    (last_lin): Linear(in_features=16, out_features=16, bias=True)
    (merge_lin): Linear(in_features=320, out_features=1, bias=True)
    (node_embeddings): NodeEmbedding(n_nodes=32, embedding_size=4)
    (block_0): CPGNN_block_v2(
      (channel_mixer): Linear(in_features=16, out_features=16, bias=True)
      (diff_derivative): Time_derivative_diffusion_product(
        (Conv_layer): GCN_diff(
          (conv1): GCNConv(16, 16)
        )
      )
    )
    (block_1): CPGNN_block_v2(
      (channel

/tmp/ipykernel_59968/3685543249.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  evecs = torch.tensor(evecs).to(device),
/home/aref/Documents/Papers/CGP-GNN/2023_TIDE: Time Derivative Diffusion for Deep Learning on Graphs/CPGNN_our_Imp/Final_Sent_Codes2/Molene/layers.py:338: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(self.diffusion_time)


Iter 0
 Epoch 0
	 train-loss: 5.809 | valid-loss: 6.349 	| valid-metric: 0.994 | lr: 0.001

				New best val_metric: 0.994. Saving model...

Iter 0
 Epoch 1
	 train-loss: 5.697 | valid-loss: 6.273 	| valid-metric: 0.988 | lr: 0.001

				New best val_metric: 0.988. Saving model...

Iter 0
 Epoch 2
	 train-loss: 5.594 | valid-loss: 6.204 	| valid-metric: 0.982 | lr: 0.001

				New best val_metric: 0.982. Saving model...

Iter 0
 Epoch 3
	 train-loss: 5.485 | valid-loss: 6.133 	| valid-metric: 0.977 | lr: 0.001

				New best val_metric: 0.977. Saving model...

Iter 0
 Epoch 4
	 train-loss: 5.335 | valid-loss: 6.055 	| valid-metric: 0.971 | lr: 0.001

				New best val_metric: 0.971. Saving model...

Iter 0
 Epoch 5
	 train-loss: 5.265 | valid-loss: 5.972 	| valid-metric: 0.964 | lr: 0.001

				New best val_metric: 0.964. Saving model...

Iter 0
 Epoch 6
	 train-loss: 5.154 | valid-loss: 5.873 	| valid-metric: 0.956 | lr: 0.001

				New best val_metric: 0.956. Saving model...

Iter 0
 Epoch

Iter 0
 Epoch 59
	 train-loss: 2.022 | valid-loss: 2.498 	| valid-metric: 0.623 | lr: 0.001

				New best val_metric: 0.623. Saving model...

Iter 0
 Epoch 60
	 train-loss: 2.012 | valid-loss: 2.462 	| valid-metric: 0.619 | lr: 0.001

				New best val_metric: 0.619. Saving model...

Iter 0
 Epoch 61
	 train-loss: 1.98 | valid-loss: 2.461 	| valid-metric: 0.619 | lr: 0.001
Iter 0
 Epoch 62
	 train-loss: 1.96 | valid-loss: 2.439 	| valid-metric: 0.616 | lr: 0.001

				New best val_metric: 0.616. Saving model...

Iter 0
 Epoch 63
	 train-loss: 1.944 | valid-loss: 2.411 	| valid-metric: 0.612 | lr: 0.001

				New best val_metric: 0.612. Saving model...

Iter 0
 Epoch 64
	 train-loss: 1.907 | valid-loss: 2.384 	| valid-metric: 0.609 | lr: 0.001

				New best val_metric: 0.609. Saving model...

Iter 0
 Epoch 65
	 train-loss: 1.899 | valid-loss: 2.355 	| valid-metric: 0.605 | lr: 0.001

				New best val_metric: 0.605. Saving model...

Iter 0
 Epoch 66
	 train-loss: 1.875 | valid-loss: 2.341 	

Iter 0
 Epoch 125
	 train-loss: 1.336 | valid-loss: 1.729 	| valid-metric: 0.519 | lr: 0.001

				New best val_metric: 0.519. Saving model...

Iter 0
 Epoch 126
	 train-loss: 1.328 | valid-loss: 1.718 	| valid-metric: 0.517 | lr: 0.001

				New best val_metric: 0.517. Saving model...

Iter 0
 Epoch 127
	 train-loss: 1.312 | valid-loss: 1.71 	| valid-metric: 0.516 | lr: 0.001

				New best val_metric: 0.516. Saving model...

Iter 0
 Epoch 128
	 train-loss: 1.314 | valid-loss: 1.709 	| valid-metric: 0.516 | lr: 0.001
Iter 0
 Epoch 129
	 train-loss: 1.308 | valid-loss: 1.703 	| valid-metric: 0.515 | lr: 0.001

				New best val_metric: 0.515. Saving model...

Iter 0
 Epoch 130
	 train-loss: 1.307 | valid-loss: 1.702 	| valid-metric: 0.515 | lr: 0.001
Iter 0
 Epoch 131
	 train-loss: 1.3 | valid-loss: 1.698 	| valid-metric: 0.514 | lr: 0.001

				New best val_metric: 0.514. Saving model...

Iter 0
 Epoch 132
	 train-loss: 1.313 | valid-loss: 1.68 	| valid-metric: 0.511 | lr: 0.001

				New b

...

Iter 9
 Epoch 1108
	 train-loss: 0.38 | valid-loss: 0.457 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1109
	 train-loss: 0.383 | valid-loss: 0.46 	| valid-metric: 0.268 | lr: 0.001
Iter 9
 Epoch 1110
	 train-loss: 0.384 | valid-loss: 0.454 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1111
	 train-loss: 0.381 | valid-loss: 0.459 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1112
	 train-loss: 0.378 | valid-loss: 0.455 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1113
	 train-loss: 0.378 | valid-loss: 0.455 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1114
	 train-loss: 0.379 | valid-loss: 0.458 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1115
	 train-loss: 0.384 | valid-loss: 0.455 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1116
	 train-loss: 0.383 | valid-loss: 0.457 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1117
	 train-loss: 0.38 | valid-loss: 0.456 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1118
	 train-loss: 0.378 | valid-loss: 0.458 	| v

Iter 9
 Epoch 1198
	 train-loss: 0.381 | valid-loss: 0.452 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1199
	 train-loss: 0.38 | valid-loss: 0.461 	| valid-metric: 0.268 | lr: 0.001
Iter 9
 Epoch 1200
	 train-loss: 0.382 | valid-loss: 0.455 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1201
	 train-loss: 0.379 | valid-loss: 0.45 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1202
	 train-loss: 0.384 | valid-loss: 0.461 	| valid-metric: 0.268 | lr: 0.001
Iter 9
 Epoch 1203
	 train-loss: 0.38 | valid-loss: 0.452 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1204
	 train-loss: 0.377 | valid-loss: 0.456 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1205
	 train-loss: 0.378 | valid-loss: 0.453 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1206
	 train-loss: 0.378 | valid-loss: 0.453 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1207
	 train-loss: 0.381 | valid-loss: 0.454 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1208
	 train-loss: 0.379 | valid-loss: 0.452 	| v

Iter 9
 Epoch 1288
	 train-loss: 0.377 | valid-loss: 0.457 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1289
	 train-loss: 0.377 | valid-loss: 0.452 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1290
	 train-loss: 0.377 | valid-loss: 0.457 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1291
	 train-loss: 0.374 | valid-loss: 0.451 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1292
	 train-loss: 0.377 | valid-loss: 0.453 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1293
	 train-loss: 0.377 | valid-loss: 0.45 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1294
	 train-loss: 0.379 | valid-loss: 0.453 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1295
	 train-loss: 0.374 | valid-loss: 0.458 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1296
	 train-loss: 0.382 | valid-loss: 0.453 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1297
	 train-loss: 0.382 | valid-loss: 0.458 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1298
	 train-loss: 0.373 | valid-loss: 0.45 	| 

Iter 9
 Epoch 1376
	 train-loss: 0.374 | valid-loss: 0.454 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1377
	 train-loss: 0.374 | valid-loss: 0.451 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1378
	 train-loss: 0.377 | valid-loss: 0.453 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1379
	 train-loss: 0.375 | valid-loss: 0.452 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1380
	 train-loss: 0.375 | valid-loss: 0.449 	| valid-metric: 0.264 | lr: 0.001
Iter 9
 Epoch 1381
	 train-loss: 0.375 | valid-loss: 0.454 	| valid-metric: 0.266 | lr: 0.001
Iter 9
 Epoch 1382
	 train-loss: 0.377 | valid-loss: 0.449 	| valid-metric: 0.264 | lr: 0.001
Iter 9
 Epoch 1383
	 train-loss: 0.375 | valid-loss: 0.458 	| valid-metric: 0.267 | lr: 0.001
Iter 9
 Epoch 1384
	 train-loss: 0.375 | valid-loss: 0.449 	| valid-metric: 0.264 | lr: 0.001
Iter 9
 Epoch 1385
	 train-loss: 0.375 | valid-loss: 0.451 	| valid-metric: 0.265 | lr: 0.001
Iter 9
 Epoch 1386
	 train-loss: 0.378 | valid-loss: 0.454 	

Iter 9
 Epoch 1458
	 train-loss: 0.372 | valid-loss: 0.455 	| valid-metric: 0.266 | lr: 0.0009000000000000001
Iter 9
 Epoch 1459
	 train-loss: 0.373 | valid-loss: 0.449 	| valid-metric: 0.264 | lr: 0.0009000000000000001
Iter 9
 Epoch 1460
	 train-loss: 0.372 | valid-loss: 0.45 	| valid-metric: 0.265 | lr: 0.0009000000000000001
Iter 9
 Epoch 1461
	 train-loss: 0.373 | valid-loss: 0.451 	| valid-metric: 0.265 | lr: 0.0009000000000000001
Iter 9
 Epoch 1462
	 train-loss: 0.371 | valid-loss: 0.449 	| valid-metric: 0.264 | lr: 0.0009000000000000001
Iter 9
 Epoch 1463
	 train-loss: 0.373 | valid-loss: 0.451 	| valid-metric: 0.265 | lr: 0.0009000000000000001
Iter 9
 Epoch 1464
	 train-loss: 0.37 | valid-loss: 0.45 	| valid-metric: 0.265 | lr: 0.0009000000000000001
Iter 9
 Epoch 1465
	 train-loss: 0.373 | valid-loss: 0.45 	| valid-metric: 0.264 | lr: 0.0009000000000000001
Iter 9
 Epoch 1466
	 train-loss: 0.372 | valid-loss: 0.448 	| valid-metric: 0.264 | lr: 0.0009000000000000001
Iter 9
 Epoch 

In [15]:
# print(res_dict['results'])
means = [round(el, 4) for el in np.average(res_dict['results'], axis=0)]
stds = [round(el, 4) for el in np.std(res_dict['results'], axis=0)]
print('means: ', means)
print('stds: ', stds)

means:  [0.2423, 0.3601, 0.4784, 0.5869, 0.6814]
stds:  [0.0526, 0.0473, 0.0398, 0.0316, 0.024]


In [13]:
res_dict['final_res'] = {
    'avg': means,
    'std': stds
}
with open(log_dir + '/results.json', 'w', encoding='utf-8') as f:
    json.dump(res_dict, f, ensure_ascii=False, indent=4)